In [ ]:
import numpy as np
import cv2 as cv

# The given video and calibration data
input_file = './chessboard.mp4'
K = np.array([[1.09072502e+03, 0.00000000e+00, 6.49201945e+02],
              [0.00000000e+00, 1.09056649e+03, 3.70281397e+02],
              [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])
dist_coeff = np.array([2.96992206e-01, -1.63709577e+00, 1.59143714e-03, 1.89137957e-03, 3.16860062e+00])
board_pattern = (10, 7)
board_cellsize = 0.025
board_criteria = cv.CALIB_CB_ADAPTIVE_THRESH + cv.CALIB_CB_NORMALIZE_IMAGE + cv.CALIB_CB_FAST_CHECK

# Open a video
video = cv.VideoCapture(input_file)
assert video.isOpened(), 'Cannot read the given input, ' + input_file

# Prepare a 3D box for simple AR
box_lower = board_cellsize * np.array([[2, 5, 0], [3, 5, 0], [3, 4, 0], [4, 4, 0], [4, 3, 0],
                                       [5, 3, 0], [5, 4, 0], [6, 4, 0], [6, 5, 0], [7, 5, 0],
                                       [7, 3, 0], [6, 3, 0], [6, 2, 0], [5, 2, 0], [5, 1, 0],
                                       [4, 1, 0], [4, 2, 0], [3, 2, 0], [3, 3, 0], [2, 3, 0]])
box_upper = board_cellsize * np.array([[2, 5, -1], [3, 5, -1], [3, 4, -1], [4, 4, -1], [4, 3, -1],
                                       [5, 3, -1], [5, 4, -1], [6, 4, -1], [6, 5, -1], [7, 5, -1],
                                       [7, 3, -1], [6, 3, -1], [6, 2, -1], [5, 2, -1], [5, 1, -1],
                                       [4, 1, -1], [4, 2, -1], [3, 2, -1], [3, 3, -1], [2, 3, -1]])

# Prepare 3D points on a chessboard
obj_points = board_cellsize * np.array([[c, r, 0] for r in range(board_pattern[1]) for c in range(board_pattern[0])])

# Run pose estimation
while True:
    # Read an image from the video
    valid, img = video.read()
    if not valid:
        break

    # Estimate the camera pose
    complete, img_points = cv.findChessboardCorners(img, board_pattern, board_criteria)
    if complete:
        ret, rvec, tvec = cv.solvePnP(obj_points, img_points, K, dist_coeff)

        # Draw the box on the image
        line_lower, _ = cv.projectPoints(box_lower, rvec, tvec, K, dist_coeff)
        line_upper, _ = cv.projectPoints(box_upper, rvec, tvec, K, dist_coeff)
        cv.polylines(img, [np.int32(line_lower)], True, (255, 0, 0), 2)
        cv.polylines(img, [np.int32(line_upper)], True, (0, 0, 255), 2)
        for b, t in zip(line_lower, line_upper):
            cv.line(img, np.int32(b.flatten()), np.int32(t.flatten()), (0, 255, 0), 2)

        # Print the camera position
        R, _ = cv.Rodrigues(rvec) # Alternative) scipy.spatial.transform.Rotation
        p = (-R.T @ tvec).flatten()
        info = f'XYZ: [{p[0]:.3f} {p[1]:.3f} {p[2]:.3f}]'
        cv.putText(img, info, (10, 25), cv.FONT_HERSHEY_DUPLEX, 0.6, (0, 255, 0))

    # Show the image and process the key event
    cv.imshow('Pose Estimation (Chessboard)', img)
    key = cv.waitKey(10)
    if key == ord(' '):
        key = cv.waitKey()
    if key == 27: # ESC
        break

video.release()
cv.destroyAllWindows()